# Data Importing And Cleaning

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
df = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/Datasets/Incidents_service.xlsx')
df.shape

(141712, 25)

In [3]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## 1.Feature Selection

In [4]:
df_new = df[[ 'count_updated', 'opened_by', 'Support_group', 'category_ID', 'ID_status','Created_by', 'active', 'impact']]
df_new.shape

(141712, 8)

In [5]:
df_new = df_new[df_new.ID_status != -100]
df_new.ID_status.value_counts()

Active                38716
New                   36407
Resolved              25751
Closed                24985
Awaiting User Info    14642
Awaiting Vendor         707
Awaiting Problem        461
Awaiting Evidence        38
Name: ID_status, dtype: int64

In [6]:
df_new = pd.get_dummies(df_new, columns=['Support_group','category_ID','ID_status','opened_by','Created_by', 'active'], drop_first=True)
print(df_new.shape)
df_new.head()

(141707, 534)


,count_updated,impact,Support_group_Group 11,Support_group_Group 12,Support_group_Group 13,Support_group_Group 14,Support_group_Group 15,Support_group_Group 16,Support_group_Group 17,Support_group_Group 18,Support_group_Group 19,Support_group_Group 2,Support_group_Group 20,Support_group_Group 21,Support_group_Group 22,Support_group_Group 23,Support_group_Group 24,Support_group_Group 25,Support_group_Group 26,Support_group_Group 27,Support_group_Group 28,Support_group_Group 29,Support_group_Group 3,Support_group_Group 30,Support_group_Group 31,Support_group_Group 32,Support_group_Group 33,Support_group_Group 34,Support_group_Group 35,Support_group_Group 36,Support_group_Group 37,Support_group_Group 38,Support_group_Group 39,Support_group_Group 4,Support_group_Group 41,Support_group_Group 43,Support_group_Group 44,Support_group_Group 45,Support_group_Group 46,Support_group_Group 47,...,Created_by_Created by 55,Created_by_Created by 56,Created_by_Created by 57,Created_by_Created by 58,Created_by_Created by 59,Created_by_Created by 6,Created_by_Created by 60,Created_by_Created by 61,Created_by_Created by 62,Created_by_Created by 64,Created_by_Created by 65,Created_by_Created by 66,Created_by_Created by 67,Created_by_Created by 68,Created_by_Created by 69,Created_by_Created by 7,Created_by_Created by 70,Created_by_Created by 71,Created_by_Created by 73,Created_by_Created by 74,Created_by_Created by 76,Created_by_Created by 78,Created_by_Created by 79,Created_by_Created by 8,Created_by_Created by 81,Created_by_Created by 82,Created_by_Created by 83,Created_by_Created by 84,Created_by_Created by 85,Created_by_Created by 86,Created_by_Created by 87,Created_by_Created by 88,Created_by_Created by 89,Created_by_Created by 91,Created_by_Created by 92,Created_by_Created by 96,Created_by_Created by 97,Created_by_Created by 98,Created_by_Created by 99,active_True
0,0,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,3,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,4,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,2,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


## 2. Scaling

In [7]:
from sklearn.preprocessing import MinMaxScaler
mx_scaler = MinMaxScaler()
final_df2 = mx_scaler.fit_transform(df_new.drop(columns = 'impact'))
final_df2 = pd.DataFrame(final_df2, columns=df_new.drop(columns = 'impact').columns)
print(final_df2.head())

   count_updated  Support_group_Group 11  ...  Created_by_Created by 99  active_True
0       0.000000                     0.0  ...                       0.0          1.0
1       0.023256                     0.0  ...                       0.0          1.0
2       0.031008                     0.0  ...                       0.0          0.0
3       0.007752                     0.0  ...                       0.0          1.0
4       0.015504                     0.0  ...                       0.0          1.0

[5 rows x 533 columns]


# Train Test Split

In [8]:
from sklearn.model_selection import train_test_split
X = final_df2
y = df_new['impact']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101, stratify= y)
print(y_train.value_counts())
print(y_test.value_counts())

2 - Medium    94030
3 - Low        2720
1 - High       2444
Name: impact, dtype: int64
2 - Medium    40300
3 - Low        1166
1 - High       1047
Name: impact, dtype: int64


# Dimensionality Reduction LDA

In [9]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA(n_components=15)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

# Algorithms

## 1. Decision Tree:

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
dt = DecisionTreeClassifier(random_state=101)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9376190812222144
[[  330   705    12]
 [  724 39135   441]
 [   14   756   396]]
              precision    recall  f1-score   support

    1 - High       0.31      0.32      0.31      1047
  2 - Medium       0.96      0.97      0.97     40300
     3 - Low       0.47      0.34      0.39      1166

    accuracy                           0.94     42513
   macro avg       0.58      0.54      0.56     42513
weighted avg       0.93      0.94      0.94     42513



## 2. Random Forest:

In [11]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators= 500, random_state= 101)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9506974337261543
[[  251   786    10]
 [  271 39763   266]
 [   10   753   403]]
              precision    recall  f1-score   support

    1 - High       0.47      0.24      0.32      1047
  2 - Medium       0.96      0.99      0.97     40300
     3 - Low       0.59      0.35      0.44      1166

    accuracy                           0.95     42513
   macro avg       0.68      0.52      0.58     42513
weighted avg       0.94      0.95      0.94     42513



## 3.Support Vector Classifier

In [12]:
from sklearn.svm import SVC
svc_clf = SVC()
svc_clf.fit(X_train,y_train)
y_pred = svc_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9532143109166608
[[  105   939     3]
 [   29 40201    70]
 [    6   942   218]]
              precision    recall  f1-score   support

    1 - High       0.75      0.10      0.18      1047
  2 - Medium       0.96      1.00      0.98     40300
     3 - Low       0.75      0.19      0.30      1166

    accuracy                           0.95     42513
   macro avg       0.82      0.43      0.48     42513
weighted avg       0.94      0.95      0.94     42513



## 4. Gausian NB

In [13]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train,y_train)
y_pred = nb.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9186131301013808
[[  350   660    37]
 [  662 37812  1826]
 [   12   263   891]]
              precision    recall  f1-score   support

    1 - High       0.34      0.33      0.34      1047
  2 - Medium       0.98      0.94      0.96     40300
     3 - Low       0.32      0.76      0.45      1166

    accuracy                           0.92     42513
   macro avg       0.55      0.68      0.58     42513
weighted avg       0.94      0.92      0.93     42513



## 5. KNN

In [14]:
from sklearn.neighbors  import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9484393009197186
[[  310   730     7]
 [  343 39537   420]
 [   18   674   474]]
              precision    recall  f1-score   support

    1 - High       0.46      0.30      0.36      1047
  2 - Medium       0.97      0.98      0.97     40300
     3 - Low       0.53      0.41      0.46      1166

    accuracy                           0.95     42513
   macro avg       0.65      0.56      0.60     42513
weighted avg       0.94      0.95      0.94     42513



#Meta Algorithms

## 1. Voting Classifier

In [15]:
from sklearn.ensemble import VotingClassifier
nb = GaussianNB()
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier(random_state=101)
# Define a list called classifier that contains the tuples (classifier_name, classifier)
classifiers = [('GaussianNB', nb),
('K Nearest Neighbours', knn),
('Classification Tree', dt)]
# Iterate over the defined list of tuples containing the classifiers
for clf_name, clf in classifiers:
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print('{:s} : {:.3f}'.format(clf_name, accuracy_score(y_test, y_pred)))

vc = VotingClassifier(estimators=classifiers)
vc.fit(X_train, y_train)
y_pred = vc.predict(X_test)
print('Voting Classifier: {:.3f}'.format(accuracy_score(y_test, y_pred)))

GaussianNB : 0.919
K Nearest Neighbours : 0.952
Classification Tree : 0.938
Voting Classifier: 0.949


## 2. Bagging Classifier

In [16]:
from sklearn.ensemble import BaggingClassifier
dt = DecisionTreeClassifier(random_state=101)
bc = BaggingClassifier(base_estimator=dt, n_estimators=300, n_jobs=-1)
bc.fit(X_train, y_train)
y_pred = bc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of Bagging Classifier: {:.3f}'.format(accuracy))

Accuracy of Bagging Classifier: 0.950


## 3. AdaBoost Classifier

In [17]:
from sklearn.ensemble import AdaBoostClassifier
dt = DecisionTreeClassifier(random_state=101)
abc = AdaBoostClassifier(base_estimator=dt, n_estimators=50)
abc.fit(X_train, y_train)
y_pred = bc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of Bagging Classifier: {:.3f}'.format(accuracy))

Accuracy of Bagging Classifier: 0.950


## 4. XGBoost Classifier

In [18]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of Bagging Classifier: {:.3f}'.format(accuracy))

Accuracy of Bagging Classifier: 0.953


# Cross Validation And Hyper Parameter Tuning

## 1. Using Randomized Search cv

In [19]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
gbm_param_grid = {'learning_rate': np.arange(0.05,1.05,.05),'n_estimators': [100, 200, 300],'max_depth': range(1,10)}
gbm = xgb.XGBClassifier()
randomized_cv = RandomizedSearchCV(estimator=gbm, param_distributions=gbm_param_grid, n_iter=25,random_state=101, cv=4, verbose=1, n_jobs=-1)
randomized_cv.fit(X_train, y_train)
print("Best parameters found: ",randomized_cv.best_params_)
print("Best Score found: ", np.sqrt(np.abs(randomized_cv.best_score_)))

Fitting 4 folds for each of 25 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 14.0min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 31.5min finished


Best parameters found:  {'n_estimators': 100, 'max_depth': 7, 'learning_rate': 0.05}
Best Score found:  0.9770745552016705


## 2. Using GridSearch CV

In [20]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
gbm_param_grid = {'learning_rate': [0.05],'n_estimators': [50, 100, 150],'max_depth': [6, 7, 8]}
gbm = xgb.XGBClassifier()
grid_cv = GridSearchCV(estimator=gbm, param_grid=gbm_param_grid, cv=4, verbose=1, n_jobs=-1)
grid_cv.fit(X_train, y_train)
print("Best parameters found: ",grid_cv.best_params_)
print("Best Score found: ", np.sqrt(np.abs(grid_cv.best_score_)))

Fitting 4 folds for each of 9 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  6.8min finished


Best parameters found:  {'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 100}
Best Score found:  0.9771519348716857
